# Dependences

In [ ]:
import torch
import torchvision
from torchvision import transforms
print('torchvision version', torchvision.__version__)     # Needs at least >= 0.8.0 to do cropping on tensors

from vision_transformer import vit_small
from vision_transformer4k import vit4k_xs
from main_dino4k import DataAugmentationDINO4K

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ViT-16 Model

In [ ]:
model = vit_small()
print("Num Parameters:", count_parameters(model))

x = torch.randn(4, 1, 256, 256)
print("1. Input Shape:", x.shape)
out = model(x)
print("2. Output Shape:", out.shape)

# ViT-256 Model

In [ ]:
model = vit4k_xs()
print("Num Parameters:", count_parameters(model))

t_tensorcrop = transforms.Compose([
    transforms.RandomCrop(14), # 14 x 14 for "global" crop, 6 x 6 for "local" crop
])

# [14 x 14] crop in a 16 x 16 grid would retain the same relative information as [224 x 224] in a 256 x 256 img
assert 224/256 == 14/16

# [6 x 6] crop in a 16 x 16 grid would retain the same relative information as [96 x 96] in a 256 x 256 img
assert 96/256 == 6/16

x_bag = torch.rand(256, 384)
print('1. For a 4K x 4K image, torch.load in 256-len sequence of 384-dim embeddings:', x_bag.shape)
x_bag = x_bag.unsqueeze(dim=0).unfold(1, 16, 16).transpose(1,2)
print('2. Reshape this sequence to be a 2D image grid (B NC W H):', x_bag.shape)
x_bag = t_tensorcrop(x_bag)
print('3. Applying 2D cropping (B NC W H):', x_bag.shape)
_ = model(x_bag)
print('4. Out:', _.shape)

In [ ]:
t_dino = DataAugmentationDINO4K(8)

x_bag = torch.randn(256, 384)
x_crops = t_dino(x_bag)
for idx, crop in enumerate(x_crops):
    print('Crop %d:' % (idx+1), crop.shape)